## Dogs v Cats super-charged!

In [1]:
# Put these at the top of every notebook, to get automatic reloading and inline plotting
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
# This file contains all the main external libs we'll use
from fastai.imports import *

from fastai.transforms import *
from fastai.conv_learner import *
from fastai.model import *
from fastai.dataset import *
from fastai.sgdr import *
from fastai.plots import *

PATH = "/datasets/dogscats/"
# arch=resnet50
sz = 299
arch = resnext50
bs=20
torch.cuda.set_device(1)

/root/miniconda3/envs/fastai/lib/python3.6/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [3]:
tfms = tfms_from_model(arch, sz, aug_tfms=transforms_side_on, max_zoom=1.1)
data = ImageClassifierData.from_paths(PATH, tfms=tfms, bs=bs, num_workers=4)
learn = ConvLearner.pretrained(arch, data, precompute=True, ps=0.5)

In [4]:
learn.fit(1e-2, 1)

learn.precompute=False
learn.fit(1e-2, 1, cycle_len=1)

# learn.unfreeze()
# lr=np.array([1e-4,1e-3,1e-2])
# learn.fit(lr, 3, cycle_len=1)

learn.save('299_all_50')
learn.load('299_all_50')

# log_preds,y = learn.TTA()
# probs = np.mean(np.exp(log_preds),0)

# accuracy_np(probs,y)

epoch      trn_loss   val_loss   accuracy                       
    0      0.042245   0.024633   0.9925    



epoch      trn_loss   val_loss   accuracy                       
    0      0.051207   0.021617   0.994     



# onxx to coreml

In [6]:
learn.precompute=False
learn.unfreeze()

In [7]:
learn.load('299_all_50')

### normalize image

In [8]:
import torch
import torch.nn as nn
import numpy as np
from onnx_coreml import convert
from torch.autograd import Variable
import torch.onnx
import torchvision
import onnx
def check_onnx_compatible(model, model_name, sz, input_names, output_names):
    dummy_input = Variable(torch.randn(3, sz, sz)).cuda()

    torch.onnx.export(model, dummy_input, \
                      model_name, input_names = input_names, output_names = output_names, verbose=True)
    
    
    # Check again by onnx
    # Load the ONNX model
    onnx_model = onnx.load(model_name)

    # Check that the IR is well formed
    onnx.checker.check_model(onnx_model)

    # Print a human readable representation of the graph
#     onnx.helper.printable_graph(onnx_model.graph)
    print("Done")
    return onnx_model

In [9]:
class ImageScale(nn.Module):
    def __init__(self): 
        super().__init__()
        self.denorminator = torch.full((3, sz, sz), 255.0, device=torch.device("cuda"))

    def forward(self, x): return torch.div(x, self.denorminator).unsqueeze(0)

In [10]:
final_model = [ImageScale()] + (list(learn.model.children())[:-1] + [nn.Softmax()])

In [11]:
final_model = nn.Sequential(*final_model)

In [12]:
model_name = "dog_vs_cat_image.onnx"

# Convert Pytorch model to onnx model & check if it is convertible
onnx_model = check_onnx_compatible(final_model, model_name, sz, input_names=['image'], output_names=['dogorcat'])

# Convert onnx model to Apple Core ML
mlmodel = convert(onnx.load(model_name), image_input_names = ['image'], mode='classifier', class_labels="labels.txt")
mlmodel.author = 'Hung Nguyen'
mlmodel.license = 'MIT'
mlmodel.short_description = 'This model takes a picture of a dog or cat and predicts its a cat or a dog'
mlmodel.input_description['image'] = 'Image of a dog or cat or something else'
mlmodel.output_description['dogorcat'] = 'Confidence and label of predicted dog or cat'
mlmodel.output_description['classLabel'] = 'Label of predicted dog or cat'

mlmodel.save(f'{model_name}.mlmodel')

graph(%image : Float(3, 299, 299)
      %1 : Float(64, 3, 7, 7)
      %2 : Float(64)
      %3 : Float(64)
      %4 : Float(64)
      %5 : Float(64)
      %6 : Long()
      %7 : Float(128, 64, 1, 1)
      %8 : Float(128)
      %9 : Float(128)
      %10 : Float(128)
      %11 : Float(128)
      %12 : Long()
      %13 : Float(128, 4, 3, 3)
      %14 : Float(128)
      %15 : Float(128)
      %16 : Float(128)
      %17 : Float(128)
      %18 : Long()
      %19 : Float(256, 128, 1, 1)
      %20 : Float(256)
      %21 : Float(256)
      %22 : Float(256)
      %23 : Float(256)
      %24 : Long()
      %25 : Float(256, 64, 1, 1)
      %26 : Float(256)
      %27 : Float(256)
      %28 : Float(256)
      %29 : Float(256)
      %30 : Long()
      %31 : Float(128, 256, 1, 1)
      %32 : Float(128)
      %33 : Float(128)
      %34 : Float(128)
      %35 : Float(128)
      %36 : Long()
      %37 : Float(128, 4, 3, 3)
      %38 : Float(128)
      %39 : Float(128)
      %40 : Float(128)
      %41 : Flo

1/189: Converting Node Type Div
2/189: Converting Node Type Unsqueeze
3/189: Converting Node Type Conv
4/189: Converting Node Type BatchNormalization
5/189: Converting Node Type Relu
6/189: Converting Node Type MaxPool
7/189: Converting Node Type Conv
8/189: Converting Node Type BatchNormalization
9/189: Converting Node Type Relu
10/189: Converting Node Type Conv
11/189: Converting Node Type BatchNormalization
12/189: Converting Node Type Relu
13/189: Converting Node Type Conv
14/189: Converting Node Type BatchNormalization
15/189: Converting Node Type Conv
16/189: Converting Node Type BatchNormalization
17/189: Converting Node Type Add
18/189: Converting Node Type Relu
19/189: Converting Node Type Conv
20/189: Converting Node Type BatchNormalization
21/189: Converting Node Type Relu
22/189: Converting Node Type Conv
23/189: Converting Node Type BatchNormalization
24/189: Converting Node Type Relu
25/189: Converting Node Type Conv
26/189: Converting Node Type BatchNormalization
27/189:

In [13]:
import coremltools

model =  coremltools.models.MLModel(f'{model_name}.mlmodel')

In [14]:
model.output_description

Features(dogorcat,classLabel)

In [15]:
model.visualize_spec

<bound method MLModel.visualize_spec of input {
  name: "image"
  shortDescription: "Image of a dog or cat or something else"
  type {
    imageType {
      width: 299
      height: 299
      colorSpace: RGB
    }
  }
}
output {
  name: "dogorcat"
  shortDescription: "Confidence and label of predicted dog or cat"
  type {
    dictionaryType {
      stringKeyType {
      }
    }
  }
}
output {
  name: "classLabel"
  shortDescription: "Label of predicted dog or cat"
  type {
    stringType {
    }
  }
}
predictedFeatureName: "classLabel"
predictedProbabilitiesName: "dogorcat"
metadata {
  shortDescription: "This model takes a picture of a dog or cat and predicts its a cat or a dog"
  author: "Hung Nguyen"
  license: "MIT"
}
>